# Assignment 5

Consider a scenario where a person visits a supermarket for shopping. S/He purchases various items in different sections such as clothing, grocery, utensils. Write an OpenMP program to process the bill parallelly in each section and display the final amount to be paid by the customer.
Analyze the time take by sequential and parallel processing.

In [7]:
import multiprocessing
import time

In [1]:
import numpy as np
import time
from numba import njit, prange

In [2]:
n_sections = 3
n_items = 1000

# Define the prices of the items in each section as random numbers
prices = np.random.rand(n_sections, n_items)

# Define the quantities of the items purchased by the customer in each section as random integers
quantities = np.random.randint(1, 10, size=(n_sections, n_items))


In [3]:
def process_bill_seq(prices, quantities):
    # Initialize the total amount to zero
    total = 0
    # Loop over the sections
    for i in range(n_sections):
        # Loop over the items in each section
        for j in range(n_items):
            # Add the product of price and quantity to the total amount
            total += prices[i, j] * quantities[i, j]
    # Return the total amount
    return total

In [4]:
@njit(parallel=True)
def process_bill_par(prices, quantities):
    # Initialize the total amount to zero
    total = 0
    # Loop over the sections in parallel using prange
    for i in prange(n_sections):
        # Loop over the items in each section
        for j in range(n_items):
            # Add the product of price and quantity to the total amount
            total += prices[i, j] * quantities[i, j]
    # Return the total amount
    return total

In [5]:
start_seq = time.time()
total_seq = process_bill_seq(prices, quantities)
end_seq = time.time()
time_seq = end_seq - start_seq

# Measure the time taken by the parallel function
start_par = time.time()
total_par = process_bill_par(prices, quantities)
end_par = time.time()
time_par = end_par - start_par

In [6]:
print(f"The total amount to be paid by the customer is {total_seq:.2f}")
print(f"The sequential function took {time_seq:.4f} seconds")
print(f"The parallel function took {time_par:.4f} seconds")
print(f"The speedup achieved by parallelization is {time_seq / time_par:.2f}")

The total amount to be paid by the customer is 7504.37
The sequential function took 0.0025 seconds
The parallel function took 2.6539 seconds
The speedup achieved by parallelization is 0.00


In [4]:
def process_section(section, items, results):
    section_total = sum(items.values())
    results[section] = section_total

if __name__ == "__main__":
    # Sample items in each section with their prices
    clothing_items = {"shirt": 20, "pants": 30, "jacket": 40}
    grocery_items = {"apple": 1, "milk": 2, "cereal": 3}
    utensils_items = {"pot": 25, "pan": 20, "utensil_set": 15}

    # Results dictionary to store section-wise totals
    results = multiprocessing.Manager().dict()

    # Sequential Processing
    start_time = time.time()

    process_section("Clothing", clothing_items, results)
    process_section("Grocery", grocery_items, results)
    process_section("Utensils", utensils_items, results)

    sequential_time = time.time() - start_time

    # Parallel Processing
    start_time = time.time()

    # Create processes for each section
    clothing_process = multiprocessing.Process(target=process_section, args=("Clothing", clothing_items, results))
    grocery_process = multiprocessing.Process(target=process_section, args=("Grocery", grocery_items, results))
    utensils_process = multiprocessing.Process(target=process_section, args=("Utensils", utensils_items, results))

    # Start the processes
    clothing_process.start()
    grocery_process.start()
    utensils_process.start()

    # Wait for all processes to finish
    clothing_process.join()
    grocery_process.join()
    utensils_process.join()

    parallel_time = time.time() - start_time

    # Calculate the total amount
    total_amount = sum(results.values())

    # Display the results
    print("Section-wise Totals:")
    for section, total in results.items():
        print(f"{section}: ${total}")

    print("\nTotal Amount to be Paid: $" + str(total_amount))

    # Display the time taken
    print("\nSequential Processing Time: {:.4f} seconds".format(sequential_time))
    print("Parallel Processing Time: {:.4f} seconds".format(parallel_time))

Section-wise Totals:
Clothing: $90
Grocery: $6
Utensils: $60

Total Amount to be Paid: $156

Sequential Processing Time: 0.0014 seconds
Parallel Processing Time: 0.1453 seconds


In [1]:
import threading
import time
import random
import os
import multiprocessing

In [9]:
clothing_items = {'Shirt': 20, 'Pants': 30, 'Sweater': 25}
grocery_items = {'Bread': 3, 'Milk': 2, 'Eggs': 1.5}
utensils_items = {'Plate': 5, 'Cup': 3, 'Fork': 1}

In [10]:
total_amount = 0.0

In [11]:
def process_section(section_name, items):
    global total_amount

    section_total = 0.0

    # Simulate the customer adding items to the cart
    cart = []
    for item, price in items.items():
        quantity = random.randint(1, 5)
        section_total += quantity * price
        cart.append(f'{item} x{quantity}')

    # Display items in the cart for the current section
    print(f"Cart in {section_name} section: {', '.join(cart)}")

    # Update the total amount
    with threading.Lock():
        total_amount += section_total

In [12]:
if __name__ == "__main__":
    # Number of parallel threads (sections)
    num_sections = 3

    # Sequential processing
    start_time = time.time()

    process_section('Clothing', clothing_items)
    process_section('Grocery', grocery_items)
    process_section('Utensils', utensils_items)

    sequential_time = time.time() - start_time

    # Display the final total amount for sequential processing
    print(f"\nSequential Total Amount to be Paid: ${total_amount:.2f}")
    print(f"Sequential Processing Time: {sequential_time:.6f} seconds\n")

    # Reset the total amount for parallel processing
    total_amount = 0.0

    # Parallel processing
    start_time = time.time()

    # Create a list of threads for each section
    threads = []

    # Clothing section processing
    clothing_thread = threading.Thread(target=process_section, args=('Clothing', clothing_items))
    threads.append(clothing_thread)

    # Grocery section processing
    grocery_thread = threading.Thread(target=process_section, args=('Grocery', grocery_items))
    threads.append(grocery_thread)

    # Utensils section processing
    utensils_thread = threading.Thread(target=process_section, args=('Utensils', utensils_items))
    threads.append(utensils_thread)

    # Start the threads
    for thread in threads:
        thread.start()

    # Wait for all threads to finish
    for thread in threads:
        thread.join()

    parallel_time = time.time() - start_time

    # Display the final total amount for parallel processing
    print(f"\nParallel Total Amount to be Paid: ${total_amount:.2f}")
    print(f"Parallel Processing Time: {parallel_time:.6f} seconds")

Cart in Clothing section: Shirt x1, Pants x2, Sweater x5
Cart in Grocery section: Bread x1, Milk x4, Eggs x3
Cart in Utensils section: Plate x3, Cup x5, Fork x4

Sequential Total Amount to be Paid: $254.50
Sequential Processing Time: 0.000000 seconds

Cart in Clothing section: Shirt x4, Pants x5, Sweater x2
Cart in Grocery section: Bread x5, Milk x1, Eggs x1
Cart in Utensils section: Plate x1, Cup x1, Fork x2

Parallel Total Amount to be Paid: $308.50
Parallel Processing Time: 0.007030 seconds
